In [14]:
import torch

from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline

from sklearn.metrics.pairwise import cosine_similarity

from datasets import load_dataset
import math
import json

import numpy as np


In [24]:
ukraine = "Po mnoha letech provokací z ruské strany zahájilo Rusko 24. února 2022 rozsáhlou invazi na Ukrajinu ve snaze ovládnout zemi během několika dní. Ukrajina se dokázala úspěšně bránit a zatlačit ruskou armádu a osvobozuje stále větší část svého území. Mnoho evropských zemí a USA odsoudili ruský útok a pomáhají Ukrajině s dovozem zbraní, humanitární pomocí a také s přijímáním mnoha uprchlíků z Ukrajiny. Na Rusko bylo uvaleno mnoho ekonomických sankcí. Rusko rovněž vede hybridní válku šířením lží po celém internetu prostřednictvím sociálních sítí, zejména v regionu střední Evropy. Mnoho politiků volá po mírových rozhovorech, které by znamenaly odevzdání některých ukrajinských území Rusku, což je pro Ukrajince nepřijatelné. Na druhou stranu ruská agrese přesvědčila neutrální země jako Finsko a Švédsko, aby vstoupily do NATO. Ukrajinský prezident Zelenskyj je od začátku invaze symbolem svobody. Na druhé straně byl ruský prezident Putin symbolem autokracie, propagandy a extremistů. Válka stále pokračuje, ruské síly vytvořily řadu zákopových linií na obranu zabraných území, což znamenalo zastavení ukrajinské ofenzívy. I když prezident Zelenskyj projevuje svou vděčnost západním zemím, stále říká, že Ukrajina dostává jen tolik zbraní, aby se bránila, ale ne tolik, aby porazila Rusko. Západní svět se bojí jaderných zbraní provozovaných Ruskem. Premiér Petr Fiala schvaluje pomoc, která je posílána Ukrajině. Byl jedním z prvních evropských politiků, kteří navštívili prezidenta Zelenského v Kyjevě na Ukrajině. Současná česká vláda je jednou z nejvíce proukrajinských a protiruských vlád na světě. 24. února 2022 zrána zaútočilo Rusko na Ukrajinu. Zvěsti o možném napadení Ukrajiny Ruskem se v médiích objevovaly několik týdnů předtím a postupně eskalovaly. Později se zjistilo, že o chystaném útoku na Ukrajinu věděl americký prezident Joe Biden i samotný prezident Ukrajiny Volodymyr Zelenskyj již v říjnu 2021. Z původního plánu Ruska provést bleskovou operaci a Ukrajinu si podmanit se stal vleklý boj, který si vyžádal esetitisíce mrtvých z řad vojáků i civilistů. Dopad má válka i na ekonomiku celé Evropy a dalších zemí. Jak to všechno začalo a proč?  Konflikt mezi Ruskem a Ukrajinou nezačal zčista jasna v únoru 2022. Samotné válce předcházely spory sahající do nedávné i daleké historie, vztahy mezi Ruskem a Ukrajinou jsou historicky napjaté. O historii Ukrajiny od Kyjevské Rusi, ruské expanzi v 16. století, součásti SSSR až po anexi Krymu v roce 2014 jsme podrobně psali zde. Ruský prezident Vladimir Putin má však na faktické historické pozadí Ukrajiny jiný názor. V červenci 2021 Putin uveřejnil esej o Ukrajině, které upřel právo na samostatnou existenci. Tvrdil, že Ukrajina nemá vlastní samostatnou historii, nemá tradici vlastní státnosti; proto Ukrajina sice může existovat jako formálně samostatný stát, ale jen v nějakém svazku s Ruskem.  Kde probíhaly největší boje Útoky ruské armády byly již od začátku roztříštěné prakticky po celé zemi. Nejtvrdší boje se odehrávaly v Mariupolu, Charkově, Kyjevě a jeho předměstí, dlouhé boje se sváděly například i o Záporožskou elektrárnu nebo ocelárny Azovstal. Masakry a zvěrstva války Jedny z nejhorších tragédií jsou spojovány s přístavním městem Mariupol Celým světem pak otřásly zprávy o masakru v Buči v Kyjevské oblasti. Společně s Irpiní a Hostomelem byla Buča trojicí míst, kde se v bitvě o Kyjev odehrály ty nejprudší boje. Ukrajinská armáda je na začátku dubna získala zpět pod svou kontrolu, ruské jednotky však po svém ústupu po sobě zanechaly děsivé svědectví o  věrstvech na Ukrajině. Válka na Ukrajině si vyžádala možná až půl milionu obětí. Většinou vojenských, které se mohou jen odhadovat, protože ani jedna strana své ztráty nezveřejňuje. Další miliony lidí ztratily své domovy a mnozí skončili v zahraničí. Uprchlická krize Svou zemi opustilo za půl roku války více než 11 milionů lidí. V České republice má vízum více než 400 tisíc Ukrajinců. Téměř pět milionů lidí se již vrátilo zpět na Ukrajinu. Přesný počet ukrajinských uprchlíků však nelze určit. Sankce Prakticky okamžitě poté, co Rusko překročilo hranice Ukrajiny, začal svět reagovat. Proti Rusku začaly evropské země, ale i USA, Austrálie nebo Kanada uvalovat nejrůznější sankce. Ty mířily proti státu, ale i ruským oligarchům. Řada zemí vypověděla své ruské diplomaty. EU se postupně shodla na celkem šesti balíčcích sankcí proti Rusku."
palestine_israel = "Konflikt mezi Izraelem a Hamásem začal 7. října 2023, kdy byla z pásma Gazy vypálena palba tisíců raket namířených na Izrael a asi 3000 ozbrojených teroristů prolomilo zeď oddělující Izrael a Gazu a zaútočilo na nedaleké vesnice, rabovali, znásilňovali a zabíjeli. Zpátky s sebou vzali přibližně 240 rukojmí zpět do Gazy. Izrael odpověděl a odřízl Gazu od vody a zásob a začal na Gazu odpalovat rakety. O několik týdnů později Izrael zahájil invazi do pásma Gazy. Hamas se snaží vyměnit rukojmí za Palestince držené v izraelských věznicích. Koncem listopadu se na několik dní konalo příměří. V Evropě a Asii se konalo mnoho protestů za palestinský lid, aby Izrael zastavil válku a pustil do Gazy humanitární pomoc. Dosud Izrael vpustil určitou pomoc od mnoha mezinárodních humanitárních organizací. Do dnes zemřelo v palestině přes deset tisíc lidí, převážně dětí. Společenská podpora izraele v České republice je velmi velká. Petr Fiala vyjádřil podporu izraelské vládě a zdůraznil právo izraelského lidu na sebeobranu.  Minulou sobotu vypukla na Blízkém východě válka mezi Izraelem a Palestinou, respektive teroristickým hnutím Hamás. Jeden z nejsložitějších konfliktů naší planety má ale kořeny v dávné minulosti. Kde se vzala vzájemná nevraživosti mezi Židy a Araby? A jak se současný konflikt vyvíjí? V zemi zaslíbené po dvou letech znovu oživl konflikt. Hamás, který sám sebe považuje za islámskou a palestinskou politickou stranu (v očích Západu se ale jedná o teroristickou organizaci), zaútočil na Izrael v sobotu 7. října v ranních hodinách. Radikálové zahájili plošný útok po souši, po vodě i ze vzduchu pomocí motorových větroňů. Kryli se přitom mohutnou raketovou palbou. Jedním z hlavních bodů útoků se stal hudební festival poblíž kibucu Re’im, kde teroristé Hamásu vraždili Izraelce i zahraniční turisty. V první den války přišlo o život nejméně 700 Izraelců. Jeruzalém okamžitě odpověděl odvetnou palbou a začal vesnice zabírané Hamásem dobývat zpět. Izraelské tajné služby mají ve světě špičkovou pověst. Podle listu Jerusalem Post ale Hamás sobotním plošným útokem domácí rozvědku překvapil. Avšak na druhé straně se objevila informace, že egyptská zpravodajská služba měla varovat izraelskou vládu, konkrétně premiéra Benjamina Netanjahua, že Hamás plánuje operaci velkého rozsahu.Avizováno to mělo být nejméně deset dní předem. Není jasné, proč avízu Netanjahu nevěnoval pozornost. Zemi navíc letos paralyzovaly protivládní protesty a pozornost se soustředila jinam. Na izraelského premiéra tak padá kritika kvůli tomu, že se nechal ukolébat pocitem, že žádné nebezpečí nehrozí. Kdo podporuje Izrael a kdo Palestinu Na stranu Izraele se podle listu Times of Israel postavilo nejméně 84 států, které Jeruzalému vyjádřily plnou podporu. Jedná se zejména o alianční partnery ze Zá padu, celou Evropskou unii, Velkou Británii, Norsko, Ukrajinu, Spojené státy nebo Kanadu (to i přesto, že mnohé z těchto zemí finančně podporovaly Palestinu). Pevným spojencem Izraele je také Česká republika. Většina arabského světa ale považuje Izrael za okupanta, který Palestince utlačuje. První v řadě partnerů Palestinců stojí Írán, jenž měl podle listu The Wall Street Journal nejenže Hamás finančně podporovat, ale dokonce s ním i celý útok naplánovat. Podporu vyjádřil radikálním útočníkům rovněž Katar, Jemen nebo Jihoafrická republika. Jedinou výjimkou jsou Spojené arabské emiráty, které už s Izraelem normalizovaly vztahy a útok Hamásu odsoudily. Co je to Palestina?     Pásmo Gazy – úzký územní pás u hranice Izraele a Egypta, kde žijí bezmála dva miliony Palestinců. Západní břeh Jordánu – nachází se na východě Izraele. Kromě židovského státu sousedí s Jordánskem, které se považuje stejně jako Egypt za spojence Izraele. Západní břeh Jordánu odděluje na jedné straně, jak název napovídá, řeka Jordán, na druhé straně oblast geograficky prostupuje městem Jeruzalém. Západní břeh Jordánu tak rozděluje hlavní město Izraele a náboženskou baštu Arabů i Židů na západní a východní Jeruzalém. Arabové obývají převážně jeho východní část."
president_elections = "Prezidentské volby se v České republice konaly v lednu 2023. První kolo vyhrál Petr Pavel, bývalý předseda Vojenského výboru NATO, Petr Pavel byl proNATO, proevropský a prozápadní kandidát.Současný český premiér projevil podporu Petru Pavlovi. Těsně na druhém místě skončil populistický kandidát, bývalý premiér Andrej Babiš, který vyjádřil nesouhlas s českou účastí ve válce na Ukrajině a s neschopností vlády starat se o lidi. Druhé kolo se konalo 27. a 28. ledna. Andrej Babiš byl odpůrce Petra Fialy. Petr Pavel vyhrál druhé kolo proti Babišovi se ziskem 58,33 % hlasů a stal se nově zvoleným prezidentem České republiky. Do funkce se ujal 9. března 2023 a nahradil Zemana. Babiš uznal Pavlovu Volební účast ve druhém kole byla mírně nad 70 %, což je nejvyšší v přímé české prezidentské volbě a nejvyšší ve všech národních českých volbách od roku 1998. Petr Fiala má s prezidentem Petrem Pavlem sdílí prozápadní hodnoty.Do druhého kola prezidentských voleb by nyní postoupili předseda hnutí ANO Andrej Babiš a někdejší vysoký představitel armády Petr Pavel, vyplývá z volebního modelu Data Collect a Kantar, jehož výsledky v neděli zveřejnila Česká televize. Babiš by podle modelu získal v prvním volebním kole 27 procent hlasů, Pavel 26,5 procenta. Třetí by skončila Danuše Nerudová s 23,5 procenta hlasů.Jiná je situace u takzvaného volebního potenciálu, což je podíl lidí, kteří volbu daného kandidáta zvažují. Z tohoto pohledu má Pavel 37,5 procenta, Nerudová 34,5 procenta a Babiš 33,5 procenta. Poslanec a předseda hnutí ANO, bývalý premiér ČR, odnikatel. Andrej Babiš je kandidátem navrženým hnutím ANO. Oficiálně oznámil svou kandidaturu na prezidenta 30. října 2022. Získal 56 podpisů od poslanců, čímž splnil zákonnou povinnost získat alespoň 20 podpisů poslanců. Jeho transparentní účet, který oznámil Úřadu byl zveřejněn 4. listopadu 2022, z tohoto účtu však není financována jeho volební kampaň, ta je financována z transparentního účtu spravovaného hnutím ANO, který byl zveřejněn ve stejné době a ze kterého jsou čerpána data v následujících grafech. Další užitečné informace o Andreji Babišovi najdete mimo jiné v online databázi Našipolitici.cz. Armádní generál ČR ve výslužbě, bývalý předseda vojenského výboru NATO. Petr Pavel svou kandidaturu oficiálně oznámil 6. září 2022. O prezidentské křeslo se uchází jako občanský kandidát. Petr Pavel sesbíral 81 329 platných podpisů občanů a stal se tak registrovaným kandidátem. Jeho kandidaturu podpořila koalice SPOLU. První transakce na transparentním účtu proběhla 7. července 2022. Další užitečné informace o Petru Pavlovi najdete mimo jiné v online databázi Našipolitici.cz.  Armádní generál ve výslužbě a bývalý předseda vojenského výboru NATO Petr Pavel narazí ve druhém kole prezidentských volbeb na Andreje Babiše. S Babišem má Pavel společné členství v KSČ, o kterém dnes bývalý vojenský důstojník otevřeně hovoří. Podporuje směřování republiky na Západ a posilování spolupráce v rámci Evropské unie nebo Severoatlantické aliance. Sloganem jeho prezidentské kampaně je „Vraťme Česku řád a klid.“ Prezidentské volby se konají v lednu. O Petru Pavlovi kolovala na internetu řada dezinformací i v následujících dnech. Jednou z nich bylo například to, že svévolně vyhlásí mobilizaci či povinnou vojenskou službu.[58][54] SMS zprávami byla rozesílána nepravdivá informace, uvádějící falešné tvrzení, že pocházejí od Petra Pavla, ve znění: „Dobry den, tady General Pavel. Touto zpravou bych Vam chtel podekovat za hlas ve volebni kampani na prezidenta. Poprosim Vas, abyste se dostavil na nejblizsi pobocku ACR, kde budete vybaven potrebnou zbroji pro mobilizaci na Ukrajine.“[59][60] Sociální sítě obletělo lživé video, které bylo manipulativně sestříháno tak, aby vyznělo, že cílem generála Pavla je zatažení Česka do války na Ukrajině, ačkoli ze skutečného videozáznamu vyplývá pravý opak. Ve čtvrtek 26. ledna falešný web uvedl, že Petr Pavel téhož dne ráno zemřel. On sám i jeho skutečný tým se proti tvrzení okamžitě ohradili. Terčem kritiky se staly i billboardy Andreje Babiše před druhým kolem volby, které nejprve hlásaly: Nezavleču Česko do války. Jsem diplomat. Ne voják.[64] V souvislosti s tím média připomněla Babišův protichůdný výrok z knihy O čem sním, když náhodou spím, který zněl: „NATO by se mělo změnit z obranného paktu na útočný.“[65] Po značné kritice tak došlo k nahrazení původních billboardů novými, tentokrát s heslem: Generál nevěří v mír. Volte mír. Volte Babiše. Ani tato verze však neunikla negativním reakcím. Oba kandidáti podnikli v rámci svých kampaní několik setkání po celé České republice."
slovakia_elections = "Slovenské parlamentní volby se konaly v září 2023 poté, co předchozí vláda Igora Matoviče prohrála hlasování o nedůvěře. Vítěz voleb, levicový populista a proruský politik Robert Fico  dokázal sestavit vládu se sociálně demokratickou stranou HLAS a Slovenskou národní stranou. Robert Fico vedl kampaň šířením dezinformací, lží a kritikou evropské vlády, Ukrajiny a myšlenek liberální demokracie. Během posledních pár let měli vyšetřovatelé plné ruce práce s odhalováním mnoha korupčních obchodů, které byly uskutečněny za Ficovy předchozí vlády. Petr Fiala s Ficem nesdílí názory na proruskou orientaci země. Po volební noci ale Ficovi pogratuloval a popřál Slovensku „dobrou vládu“ v naději, že Česká a Slovenská republika udrží své skvělé vztahy, což není zcela jisté. Volby nabídly soupeření řady bývalých předsedů vlád. A také mnoho stran, které se v průzkumech pohybovaly kolem prahu pro vstup do parlamentu. Slovensko se dlouhodobě potýká s vysokými podíly propadlých hlasů.Vysoko se v průzkumech objevoval Směr sociální demokracie v čele s Robertem Ficem, který nakonec volby skutečně vyhrál. Premiérem byl již v letech 2006–2010 a 2012–2018. Také Peter Pellegrini, lídr Hlasu – sociální demokracie, již má za sebou působení v čele vlády. V roce 2020 založil Hlas spolu s dalšími poslanci původně zvolenými za Směr Eduard Heger se v březnu 2023 stal předsedou strany Demokraté, z druhého místa kryl záda lékařce Andree Letanovské. S personálními změnami mířila do voleb koalice OĽANO a přátelé. Vedla ji předsedkyně Žilinského kraje Erika Jurinová. Až na posledním, 150., místě kandidoval Igor Matovič. Blízko konce seznamu byla i Veronika Remišová, místopředsedkyně Matovičovy i Hegerovy vlády a šéfka strany Za lidi. Místopředseda Evropského parlamentu Michal Šimečka vedl Progresivní Slovensko. Strana, v níž dříve byla Zuzana Čaputová místopředsedkyní, se v roce 2020 v koalici jen o tisíc hlasů nedostala do parlamentu. O politický návrat se v uskupení Modrí, Most – Híd pokoušel Mikuláš Dzurinda, který již řídil vládu mezi lety 1998 a 2006. Kandidovaly také další strany z Matovičovy i Hegerovy vlády. Jsme rodina byla opět vedena předsedou parlamentu Borisem Kollárem. Nedávno čelil snaze o odvolání kvůli výrokům o udeření bývalé partnerky, ale kvůli nízké účasti poslanců nebyla mimořádná schůze opakovaně zahájena. Lídrem SaS byl opět Richard Sulík, který svou demisí podtrhl vládní nestabilitu a zahájil exodus ministrů. Z aktuální opozice chtěla svou pozici v parlamentu udržet i strana Kotlebovci – Lidová strana Naše Slovensko. Lídrem listiny je Marek Kotleba, bratr dlouholetého předsedy Mariana Kotleby. Ten v roce 2022 přišel o mandát po odsouzení za rozdávání šeků s nacistickou symbolikou. V době voleb mu ještě probíhal zkušební odklad půlročního podmíněného trestu. Někteří dřívější představitelé strany dnes působí v krajně pravicové Republice, kterou vedl europoslanec Milan Uhrík. Proč byly volby předčasné V létě 2022 vládní strana Svoboda a solidarita (SaS) vypověděla koaliční smlouvu a dala premiéru Eduardu Hegerovi (OĽANO) ultimátum. Buď vládu opustí ministr financí Igor Matovič, nebo její ministři podají demisi. Jenže Matovič, sám předseda OĽANO a bývalý premiér, v kabinetu zůstal. Vláda pokračovala jako menšinová, SaS v prosinci vyvolala hlasování o vyslovení nedůvěry vládě a podpořila její pád. Matovič z vlády odešel až o týden později, když SaS podpořila navržený rozpočet pro další rok. Vláda bez důvěry na Slovensku zůstává ve funkci do jmenování nové vlády. Mezitím se však nesmí pouštět do zásadních otázek vnitřní, zahraniční či hospodářské politiky. Obsazování míst státních funkcionářů může provádět jen s prezidentovým souhlasem. Volby. Do odvolané vlády není možné přidávat nové členy, Hegerův kabinet se dalšími demisemi jen ztenčoval. V květnu prezidentka Zuzana Čaputová jmenovala první slovenskou „úřednickou“ vládu. Předsedou se stal Ľudovít Ódor, předtím viceguvernér Národní banky Slovenska. Kabinet nestraníků nezískal důvěru parlamentu, také tedy vládne s omezenými pravomocemi. Dříve se předčasné volby vyvolávaly jednorázovými ústavními zákony. Nyní slovenská ústava, podobně jako česká, umožňuje rozpuštění na návrh tří pětin (90) poslanců. Novinku poslanci nachystali a hned použili právě pro volby v roce 2023. Opoziční Směr-SD vedený Robertem Ficem chtěl rozšířit ústavu i o možnost odstavování vlád prostřednictvím celostátních referend. Jenže referendum, kterým by Slováci o takovou změnu žádali, neplatilo kvůli nízké účasti. Slovenský parlament tvoří jediná komora. NR SR má 150 členů volených na čtyři roky poměrným zastoupením."

In [27]:
texts_dict = {
    'ukraine': ukraine,
    'palestine_israel': palestine_israel,
    'president_elections': president_elections,
    'slovakia_elections': slovakia_elections
}



In [41]:
text1 = "Po mnoha letech provokací z ruské strany zahájilo Rusko 24. února 2022 rozsáhlou invazi na Ukrajinu ve snaze ovládnout zemi během několika dní. Ukrajina se dokázala úspěšně bránit a zatlačit ruskou armádu a osvobozuje stále větší část svého území. Mnoho evropských zemí a USA odsoudili ruský útok a pomáhají Ukrajině s dovozem zbraní, humanitární pomocí a také s přijímáním mnoha uprchlíků z Ukrajiny. Na Rusko bylo uvaleno mnoho ekonomických sankcí. Rusko rovněž vede hybridní válku šířením lží po celém internetu prostřednictvím sociálních sítí, zejména v regionu střední Evropy. Mnoho politiků volá po mírových rozhovorech, které by znamenaly odevzdání některých ukrajinských území Rusku, což je pro Ukrajince nepřijatelné. Na druhou stranu ruská agrese přesvědčila neutrální země jako Finsko a Švédsko, aby vstoupily do NATO. Ukrajinský prezident Zelenskyj je od začátku invaze symbolem svobody. Na druhé straně byl ruský prezident Putin symbolem autokracie, propagandy a extremistů. Válka stále pokračuje, ruské síly vytvořily řadu zákopových linií na obranu zabraných území, což znamenalo zastavení ukrajinské ofenzívy. I když prezident Zelenskyj projevuje svou vděčnost západním zemím, stále říká, že Ukrajina dostává jen tolik zbraní, aby se bránila, ale ne tolik, aby porazila Rusko. Západní svět se bojí jaderných zbraní provozovaných Ruskem. Premiér Petr Fiala schvaluje pomoc, která je posílána Ukrajině. Byl jedním z prvních evropských politiků, kteří navštívili prezidenta Zelenského v Kyjevě na Ukrajině. Současná česká vláda je jednou z nejvíce proukrajinských a protiruských vlád na světě."
text2 = "Konflikt mezi Izraelem a Hamásem začal 7. října 2023, kdy byla z pásma Gazy vypálena palba tisíců raket namířených na Izrael a asi 3000 ozbrojených teroristů prolomilo zeď oddělující Izrael a Gazu a zaútočilo na nedaleké vesnice, rabovali, znásilňovali a zabíjeli. Zpátky s sebou vzali přibližně 240 rukojmí zpět do Gazy. Izrael odpověděl a odřízl Gazu od vody a zásob a začal na Gazu odpalovat rakety. O několik týdnů později Izrael zahájil invazi do pásma Gazy. Hamas se snaží vyměnit rukojmí za Palestince držené v izraelských věznicích. Koncem listopadu se na několik dní konalo příměří. V Evropě a Asii se konalo mnoho protestů za palestinský lid, aby Izrael zastavil válku a pustil do Gazy humanitární pomoc. Dosud Izrael vpustil určitou pomoc od mnoha mezinárodních humanitárních organizací. Do dnes zemřelo v palestině přes deset tisíc lidí, převážně dětí. Společenská podpora izraele v České republice je velmi velká. Petr Fiala vyjádřil podporu izraelské vládě a zdůraznil právo izraelského lidu na sebeobranu."
text3 = "Prezidentské volby se v České republice konaly v lednu 2023. První kolo vyhrál Petr Pavel, bývalý předseda Vojenského výboru NATO, Petr Pavel byl proNATO, proevropský a prozápadní kandidát.Současný český premiér projevil podporu Petru Pavlovi. Těsně na druhém místě skončil populistický kandidát, bývalý premiér Andrej Babiš, který vyjádřil nesouhlas s českou účastí ve válce na Ukrajině a s neschopností vlády starat se o lidi. Druhé kolo se konalo 27. a 28. ledna. Andrej Babiš byl odpůrce Petra Fialy. Petr Pavel vyhrál druhé kolo proti Babišovi se ziskem 58,33 % hlasů a stal se nově zvoleným prezidentem České republiky. Do funkce se ujal 9. března 2023 a nahradil Zemana. Babiš uznal Pavlovu Volební účast ve druhém kole byla mírně nad 70 %, což je nejvyšší v přímé české prezidentské volbě a nejvyšší ve všech národních českých volbách od roku 1998. Petr Fiala má s prezidentem Petrem Pavlem sdílí prozápadní hodnoty. "
text4 = "Slovenské parlamentní volby se konaly v září 2023 poté, co předchozí vláda Igora Matoviče prohrála hlasování o nedůvěře. Vítěz voleb, levicový populista a proruský politik Robert Fico  dokázal sestavit vládu se sociálně demokratickou stranou HLAS a Slovenskou národní stranou. Robert Fico vedl kampaň šířením dezinformací, lží a kritikou evropské vlády, Ukrajiny a myšlenek liberální demokracie. Během posledních pár let měli vyšetřovatelé plné ruce práce s odhalováním mnoha korupčních obchodů, které byly uskutečněny za Ficovy předchozí vlády. Petr Fiala s Ficem nesdílí názory na proruskou orientaci země. Po volební noci ale Ficovi pogratuloval a popřál Slovensku „dobrou vládu“ v naději, že Česká a Slovenská republika udrží své skvělé vztahy, což není zcela jisté."

In [42]:
contexts = {
    'ukraine': text1,
    'palestine_israel': text2,
    'president_elections': text3,
    'slovakia_elections': text4
}



In [43]:
q_ukraine = [
    "Kdy začala konflikt na Ukrajině a jaký byl jeho původ?",
    "Jaký byl vliv anexe Krymu na eskalaci konfliktu?",
    "Které země podporují Ukrajinu a které podporují separatisty?",
    "Jaká je aktuální humanitární situace v postižených oblastech?",
    "Jaké jsou diplomatické snahy o řešení konfliktu a mírový proces?",
    "Jaký je vliv války na civilní obyvatelstvo v dané oblasti?",
    "Které mezinárodní organizace se aktivně angažují ve snaze ukončit konflikt?",
    "Jaké jsou následky války na ekonomiku Ukrajiny a okolních zemí?",
    "Existuje nějaký plán obnovy a rekonstrukce po skončení konfliktu?",
    "Jaký je postoj veřejnosti v různých částech světa k ukrajinskému konfliktu?"
]

# Válka v Palestině
q_palestine = [
    "Jak vznikl současný konflikt mezi Izraelem a Palestinou?",
    "Které faktory přispěly k nedávným násilnostem a eskalaci napětí?",
    "Jaké jsou hlavní body sporu mezi Izraelem a Palestinci?",
    "Jaké jsou důsledky konfliktu na civilní obyvatelstvo na obou stranách?",
    "Jaká je role mezinárodní komunity při řešení izraelsko-palestinského konfliktu?",
    "Existuje nějaký mírový plán nebo jednání mezi stranami konfliktu?",
    "Jaký je postoj arabských států k izraelsko-palestinskému konfliktu?",
    "Které mezinárodní organizace monitorují situaci a poskytují humanitární pomoc?",
    "Jak se promítá konflikt do regionální stability na Blízkém východě?",
    "Jaký má konflikt vliv na vnímání Izraele a Palestiny ve světě?"
]

# České prezidentské volby
q_president = [
    "Kdo jsou hlavní kandidáti v nadcházejících českých prezidentských volbách?",
    "Jaké jsou hlavní politické témata diskutovaná během volební kampaně?",
    "Jaké jsou klíčové výzvy, kterým bude český prezident čelit v nadcházejícím období?",
    "Jaký je postoj veřejnosti k jednotlivým kandidátům a jejich programům?",
    "Jaký je vliv mediálního pokrytí na průběh volební kampaně?",
    "Jakým způsobem probíhá volební proces a jaký je očekávaný volební účast?",
    "Které politické strany nebo hnutí podporují jednotliví kandidáti?",
    "Jaká je role prezidenta v českém politickém systému a jaký má pravomoci?",
    "Jaké jsou předpokládané důsledky výsledků prezidentských voleb pro politickou scénu v České republice?",
    "Jaký je vliv prezidentských voleb na vnímání České republiky v mezinárodním kontextu?"
]

# Slovenské parlamentní volby
q_slovakia = [
    "Kdo jsou hlavní politické strany a kandidáti ve slovenských parlamentních volbách?",
    "Jaké jsou klíčové politické otázky, které dominují volební kampani?",
    "Jaký je vztah mezi Slovenskem a Evropskou unií v kontextu voleb?",
    "Jaký je postoj veřejnosti k jednotlivým politickým stranám a lídrům?",
    "Jaký je politický systém na Slovensku a jak funguje parlamentní volba?",
    "Které otázky týkající se ekonomiky a sociální politiky jsou ve volbách klíčové?",
    "Jaká je role Slovenska v regionálním a mezinárodním kontextu, a jak ovlivňují volby tuto pozici?",
    "Jaká je očekávaná volební účast a jaké jsou faktory, které ji mohou ovlivnit?",
    "Jaký je postoj kandidátů k otázkám životního prostředí a udržitelnosti?",
    "Jak by výsledek voleb mohl ovlivnit vnitřní politické a hospodářské události ve Slovenské republice?"
]

In [44]:
model_name = "ufal/robeczech-base" 

In [45]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [46]:
model = AutoModelForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at ufal/robeczech-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [47]:
feature_extraction_pipeline = pipeline('feature-extraction', model=model_name, tokenizer=tokenizer)  # 'device=0' for GPU, remove for CPU

In [10]:
json_file_path = "/home/martin/Coding/DAS-political-confesions/embedding/stopwords-cs.json"
# Open and load the JSON file
with open(json_file_path, "r", encoding="utf-8") as file:
    stopwords = json.load(file)
    
special_characters_mapping = {'š': 's', 'č': 'c', 'ě': 'e', 'ž': 'z', 'ř': 'r', 'ý': 'y', 'á': 'a', 'í': 'i', 'é': 'e', 'ň': 'n'}
special_characters_mapping.update({char.upper(): replacement.upper() for char, replacement in special_characters_mapping.items()})

def preprocess_text(text, stopwords):
    
#     for special_char, replacement in special_characters_mapping.items():
#         text = text.replace(special_char, replacement)
    
    words = text.split()

#     words = [word[0:4] for word in words if len(word) > 7]
    
    filtered_words = [word for word in words if word.lower() not in stopwords]
    return ' '.join(filtered_words)

In [48]:
embeddings = {}
for key, text in contexts.items():
    
#     text = preprocess_text(text, stopwords)
    
    chunk_size = 50  # Set your desired chunk size

    # Split the text into non-overlapping chunks
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    # Initialize an empty list to store chunk embeddings
    chunk_embeddings = []

    # Process each chunk and obtain embeddings
    for chunk in chunks:
        chunk_embedding = feature_extraction_pipeline(chunk)
        chunk_embedding = np.mean(chunk_embedding[0], axis=0)
        chunk_embeddings.append(chunk_embedding)

    # Aggregate embeddings (e.g., by averaging)
    aggregated_embedding = np.mean(chunk_embeddings, axis=0)
    aggregated_embedding = np.array(aggregated_embedding).reshape(1, -1)
    embeddings[key] = aggregated_embedding
    

In [36]:

question_embeddings = {}
for q in q_slovakia:
    q = preprocess_text(q, stopwords)
#     question = preprocess_text(question, stopwords)
    emb_q = feature_extraction_pipeline(q)
    emb_q = np.mean(emb_q[0], axis=0)
    emb_q = np.array(emb_q).reshape(1, -1)
    question_embeddings[q] = emb_q
    


In [37]:
question_classes = {}

for q, emb_q in question_embeddings.items():
    max_sim = -1
    for i,(t, emb_t) in enumerate(embeddings.items()):
        sim = cosine_similarity(emb_q, emb_t)
        if sim > max_sim:
            question_classes[q] = i
            max_sim = sim

In [38]:
for key, value in question_classes.items():
    print(key, ': ', value)

hlavní politické strany kandidáti slovenských parlamentních volbách? :  2
Jaké klíčové politické otázky, dominují volební kampani? :  3
Jaký vztah Slovenskem Evropskou unií kontextu voleb? :  3
Jaký postoj veřejnosti jednotlivým politickým stranám lídrům? :  3
Jaký politický systém Slovensku funguje parlamentní volba? :  2
otázky týkající ekonomiky sociální politiky volbách klíčové? :  3
Jaká role Slovenska regionálním mezinárodním kontextu, ovlivňují volby pozici? :  3
Jaká očekávaná volební účast jaké faktory, ovlivnit? :  3
Jaký postoj kandidátů otázkám životního prostředí udržitelnosti? :  3
výsledek voleb ovlivnit vnitřní politické hospodářské události Slovenské republice? :  3


In [39]:
import pickle

In [18]:
for key,value in embeddings.items():

{'Po mnoha letech provokací z ruské strany zahájilo Rusko 24. února 2022 rozsáhlou invazi na Ukrajinu ve snaze ovládnout zemi během několika dní. Ukrajina se dokázala úspěšně bránit a zatlačit ruskou armádu a osvobozuje stále větší část svého území. Mnoho evropských zemí a USA odsoudili ruský útok a pomáhají Ukrajině s dovozem zbraní, humanitární pomocí a také s přijímáním mnoha uprchlíků z Ukrajiny. Na Rusko bylo uvaleno mnoho ekonomických sankcí. Rusko rovněž vede hybridní válku šířením lží po celém internetu prostřednictvím sociálních sítí, zejména v regionu střední Evropy. Mnoho politiků volá po mírových rozhovorech, které by znamenaly odevzdání některých ukrajinských území Rusku, což je pro Ukrajince nepřijatelné. Na druhou stranu ruská agrese přesvědčila neutrální země jako Finsko a Švédsko, aby vstoupily do NATO. Ukrajinský prezident Zelenskyj je od začátku invaze symbolem svobody. Na druhé straně byl ruský prezident Putin symbolem autokracie, propagandy a extremistů. Válka stál

In [49]:
with open('contexts_embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)